In [1]:
# =============================================================================
# MODEL PERSISTENCE & TRAINING OPTIMIZATION
# =============================================================================
# Colab notebook for teaching saving, checkpoints, early stopping, overfitting
# Uses IMDB sentiment analysis throughout
# =============================================================================

# =============================================================================
# CELL 1: SETUP & LOAD DATA
# =============================================================================


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import numpy as np
import pickle
import os

print("✅ Libraries imported!")

# Load IMDB dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.imdb.load_data(num_words=5000)

# Use subset for faster training
x_train, y_train = x_train[:20000], y_train[:20000]
x_test, y_test = x_test[:5000], y_test[:5000]

# Pad sequences using Keras
x_train = keras.utils.pad_sequences(x_train, maxlen=100)
x_test = keras.utils.pad_sequences(x_test, maxlen=100)

# Create tokenizer from IMDB word_index
word_index = keras.datasets.imdb.get_word_index()
tokenizer = keras.preprocessing.text.Tokenizer(num_words=5000)
tokenizer.word_index = word_index

# Helper to show reviews
index_to_word = {idx + 3: word for word, idx in word_index.items()}
index_to_word.update({0: '<PAD>', 1: '<START>', 2: '<UNKNOWN>'})

def show_review(index):
    """Show a review in readable text."""
    # Get original sequence from dataset
    (x_orig, y_orig), _ = keras.datasets.imdb.load_data(num_words=5000)
    text = ' '.join([index_to_word.get(num, '?') for num in x_orig[index]])
    sentiment = "POSITIVE 😊" if y_orig[index] == 1 else "NEGATIVE 😞"
    print(f"\n📝 Review #{index} ({sentiment})")
    print("-" * 50)
    print(text[:500] + "..." if len(text) > 500 else text)

print(f"\n✅ Data loaded!")
print(f"   Training samples: {len(x_train)}")
print(f"   Test samples: {len(x_test)}")
print(f"   Sequence length: {x_train.shape[1]} words")

print("\n📚 Sample reviews:")
show_review(0)
show_review(1)

/Users/kyle-anthonyhay/Documents/CODE/Revature-Training/Ai-Engineering/December/venv/lib/python3.12/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


✅ Libraries imported!

✅ Data loaded!
   Training samples: 20000
   Test samples: 5000
   Sequence length: 100 words

📚 Sample reviews:

📝 Review #0 (POSITIVE 😊)
--------------------------------------------------
<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNKNOWN> is an amazing actor and now the same being director <UNKNOWN> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNKNOWN> ...

📝 Review #1 (NEGATIVE 😞)
--------------------------------------------------
<START> big hair big <UNKNOWN> bad music and a giant safety <UNKNOWN> these are the words to best describe this terrible movie i love cheesy horror movies and i've seen hundreds but this had got to be on of

In [2]:
# =============================================================================
# CELL 2: THREE FORMATS EXPLAINED
# =============================================================================

print("""
=============================================================================
THREE WAYS TO SAVE A MODEL
=============================================================================

1. Keras Format (.keras) - RECOMMENDED
   └── my_model.keras          ← Everything in one file

   Use for: Default choice, full model save

2. H5 Format (.h5) - LEGACY
   └── my_model.h5             ← Everything in one file

   Use for: Older code compatibility

3. Weights Only (.weights.h5)
   └── my_weights.weights.h5   ← Just the learned numbers

   Use for: Transfer learning
   ⚠️  Must rebuild architecture before loading!

4. Export for Serving
   └── my_model_export/        ← Folder for TFServing/TFLite

   Use for: Production deployment

=============================================================================
""")



THREE WAYS TO SAVE A MODEL

1. Keras Format (.keras) - RECOMMENDED
   └── my_model.keras          ← Everything in one file

   Use for: Default choice, full model save

2. H5 Format (.h5) - LEGACY
   └── my_model.h5             ← Everything in one file

   Use for: Older code compatibility

3. Weights Only (.weights.h5)
   └── my_weights.weights.h5   ← Just the learned numbers

   Use for: Transfer learning
   ⚠️  Must rebuild architecture before loading!

4. Export for Serving
   └── my_model_export/        ← Folder for TFServing/TFLite

   Use for: Production deployment




In [3]:
# =============================================================================
# CELL 3: TRAIN MODEL
# =============================================================================

print("="*60)
print("TRAINING SENTIMENT MODEL")
print("="*60)

model = keras.Sequential([
    layers.Embedding(5000, 32, input_length=100, name='embedding'),
    layers.LSTM(32, dropout=0.2, recurrent_dropout=0.2, name='lstm'),
    layers.Dense(1, activation='sigmoid', name='output')
], name='sentiment_model')

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print("\n📊 Model Architecture:")
model.summary()

print("\n🏋️ Training...")
history = model.fit(
    x_train, y_train,
    epochs=3,
    batch_size=128,
    validation_split=0.2,
    verbose=1
)

test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(f"\n✅ Training complete! Test accuracy: {test_acc:.2%}")

TRAINING SENTIMENT MODEL

📊 Model Architecture:


/Users/kyle-anthonyhay/Documents/CODE/Revature-Training/Ai-Engineering/December/venv/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:100: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sentiment_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)


🏋️ Training...
Epoch 1/3
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - accuracy: 0.6953 - loss: 0.5831 - val_accuracy: 0.8177 - val_loss: 0.4111
Epoch 2/3
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.8317 - loss: 0.3958 - val_accuracy: 0.8382 - val_loss: 0.3728
Epoch 3/3
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.8586 - loss: 0.3429 - val_accuracy: 0.8225 - val_loss: 0.4021

✅ Training complete! Test accuracy: 80.46%


In [4]:
# =============================================================================
# CELL 4: SAVE MODEL - 3 WAYS
# =============================================================================

print("="*60)
print("SAVING MODEL - THREE WAYS")
print("="*60)

os.makedirs('saved_models', exist_ok=True)

# Method 1: Keras format (recommended)
model.save('saved_models/sentiment_model.keras')
print("\n✅ Keras format saved: saved_models/sentiment_model.keras")

# Method 2: H5 format (legacy)
model.save('saved_models/sentiment_model.h5')
print("✅ H5 format saved: saved_models/sentiment_model.h5")

# Method 3: Weights only
model.save_weights('saved_models/sentiment_weights.weights.h5')
print("✅ Weights saved: saved_models/sentiment_weights.weights.h5")

# Method 4: Export for TFServing/TFLite
model.export('saved_models/sentiment_export')
print("✅ Exported: saved_models/sentiment_export/")

print("\n📁 Files created:")
!ls -lh saved_models/

SAVING MODEL - THREE WAYS

✅ Keras format saved: saved_models/sentiment_model.keras
✅ H5 format saved: saved_models/sentiment_model.h5
✅ Weights saved: saved_models/sentiment_weights.weights.h5
INFO:tensorflow:Assets written to: saved_models/sentiment_export/assets


INFO:tensorflow:Assets written to: saved_models/sentiment_export/assets


Saved artifact at 'saved_models/sentiment_export'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 100), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  4971852624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4971846480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4971852240: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4971854544: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4971852432: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4971854352: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4971855120: TensorSpec(shape=(), dtype=tf.resource, name=None)
✅ Exported: saved_models/sentiment_export/

📁 Files created:
total 17816
drwxr-xr-x@ 6 kyle-anthonyhay  staff   192B Dec 30 13:06 sentiment_export
-rw-r--r--@ 1 kyle-anthonyhay  staff   2.0M Dec 30 13:06 sentiment_model.h5
-rw-r--r--@ 1 kyle-anthonyhay  st

In [5]:
# =============================================================================
# CELL 5: LOAD AND PREDICT
# =============================================================================

print("="*60)
print("LOADING MODEL & PREDICTING NEW TEXT")
print("="*60)

# Load model
loaded_model = keras.models.load_model('saved_models/sentiment_model.keras')
print("✅ Model loaded!")

def predict_sentiment(text):
    """Predict sentiment of any text."""
    sequence = tokenizer.texts_to_sequences([text])
    padded = keras.utils.pad_sequences(sequence, maxlen=100)
    prob = loaded_model.predict(padded, verbose=0)[0][0]
    return prob, "POSITIVE 😊" if prob > 0.5 else "NEGATIVE 😞"

# Test predictions
test_reviews = [
    "This movie was amazing! I loved it.",
    "Terrible film. Waste of time.",
    "It was okay, nothing special.",
]

print("\n🔮 Predictions:\n")
for review in test_reviews:
    prob, sentiment = predict_sentiment(review)
    print(f"📝 \"{review}\"")
    print(f"   → {prob:.1%} → {sentiment}\n")

LOADING MODEL & PREDICTING NEW TEXT
✅ Model loaded!

🔮 Predictions:

📝 "This movie was amazing! I loved it."
   → 93.5% → POSITIVE 😊

📝 "Terrible film. Waste of time."
   → 91.2% → POSITIVE 😊

📝 "It was okay, nothing special."
   → 86.4% → POSITIVE 😊



In [6]:
# =============================================================================
# CELL 6: SAVE/LOAD TOKENIZER
# =============================================================================

print("="*60)
print("SAVING TOKENIZER")
print("="*60)

# Save tokenizer
with open('saved_models/tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)
print("✅ Tokenizer saved!")

# Load tokenizer
with open('saved_models/tokenizer.pkl', 'rb') as f:
    loaded_tokenizer = pickle.load(f)
print("✅ Tokenizer loaded!")

# Test
test_text = "great movie"
print(f"\n📖 Test: \"{test_text}\" → {loaded_tokenizer.texts_to_sequences([test_text])}")

print("\n⚠️  Always save tokenizer with model!")

SAVING TOKENIZER
✅ Tokenizer saved!
✅ Tokenizer loaded!

📖 Test: "great movie" → [[84, 17]]

⚠️  Always save tokenizer with model!


In [7]:
# =============================================================================
# CELL 7: CHECKPOINT - SAVE BEST ONLY
# =============================================================================

print("="*60)
print("CHECKPOINT: SAVE BEST ONLY")
print("="*60)

os.makedirs('checkpoints', exist_ok=True)

model_ckpt = keras.Sequential([
    layers.Embedding(5000, 32, input_length=100),
    layers.LSTM(32, dropout=0.2, recurrent_dropout=0.2),
    layers.Dense(1, activation='sigmoid')
])
model_ckpt.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

checkpoint_best = ModelCheckpoint(
    'checkpoints/best_model.keras',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

print("🏋️ Training (watch 'saving model' messages)...\n")
model_ckpt.fit(
    x_train, y_train,
    epochs=5,
    batch_size=128,
    validation_split=0.2,
    callbacks=[checkpoint_best]
)

print("\n✅ Best model saved: checkpoints/best_model.keras")

CHECKPOINT: SAVE BEST ONLY
🏋️ Training (watch 'saving model' messages)...

Epoch 1/5


/Users/kyle-anthonyhay/Documents/CODE/Revature-Training/Ai-Engineering/December/venv/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:100: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


124/125 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.6089 - loss: 0.6563
Epoch 1: val_accuracy improved from None to 0.79675, saving model to checkpoints/best_model.keras

Epoch 1: finished saving model to checkpoints/best_model.keras
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - accuracy: 0.6995 - loss: 0.5889 - val_accuracy: 0.7968 - val_loss: 0.4508
Epoch 2/5
123/125 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.8281 - loss: 0.4119
Epoch 2: val_accuracy improved from 0.79675 to 0.83200, saving model to checkpoints/best_model.keras

Epoch 2: finished saving model to checkpoints/best_model.keras
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.8321 - loss: 0.4013 - val_accuracy: 0.8320 - val_loss: 0.3829
Epoch 3/5
123/125 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.8618 - loss: 0.3427
Epoch 3: val_accuracy did not improve from 0.83200
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.8567 - loss: 0.3474 - val_accuracy: 0.8183 - val_loss: 0.4021
Epoch 4/5
123/125 ━━

In [8]:
# =============================================================================
# CELL 8: CHECKPOINT - SAVE EVERY EPOCH
# =============================================================================

print("="*60)
print("CHECKPOINT: SAVE EVERY EPOCH")
print("="*60)

model_all = keras.Sequential([
    layers.Embedding(5000, 32, input_length=100),
    layers.LSTM(32, dropout=0.2, recurrent_dropout=0.2),
    layers.Dense(1, activation='sigmoid')
])
model_all.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

checkpoint_all = ModelCheckpoint(
    'checkpoints/model_epoch{epoch:02d}_acc{val_accuracy:.3f}.keras',
    save_best_only=False,
    verbose=1
)

print("🏋️ Training (saving every epoch)...\n")
model_all.fit(
    x_train, y_train,
    epochs=3,
    batch_size=128,
    validation_split=0.2,
    callbacks=[checkpoint_all]
)

print("\n📁 All checkpoints:")
!ls -la checkpoints/model_epoch*

CHECKPOINT: SAVE EVERY EPOCH
🏋️ Training (saving every epoch)...

Epoch 1/3
124/125 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.5963 - loss: 0.6647
Epoch 1: saving model to checkpoints/model_epoch01_acc0.793.keras

Epoch 1: finished saving model to checkpoints/model_epoch01_acc0.793.keras
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - accuracy: 0.6808 - loss: 0.6041 - val_accuracy: 0.7933 - val_loss: 0.4566
Epoch 2/3
124/125 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.8256 - loss: 0.4151
Epoch 2: saving model to checkpoints/model_epoch02_acc0.822.keras

Epoch 2: finished saving model to checkpoints/model_epoch02_acc0.822.keras
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.8294 - loss: 0.4030 - val_accuracy: 0.8217 - val_loss: 0.3999
Epoch 3/3
124/125 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.8529 - loss: 0.3545
Epoch 3: saving model to checkpoints/model_epoch03_acc0.829.keras

Epoch 3: finished saving model to checkpoints/model_epoch03_acc0.829.keras
125/125 ━━━━━━

In [9]:
# =============================================================================
# CELL 9: RESUME TRAINING
# =============================================================================

print("="*60)
print("RESUME INTERRUPTED TRAINING")
print("="*60)

print("📂 Loading checkpoint...")
resumed_model = keras.models.load_model('checkpoints/best_model.keras')
print("✅ Loaded!")

print("\n🏋️ Resuming from epoch 3 to 6...")
resumed_model.fit(
    x_train, y_train,
    initial_epoch=3,
    epochs=6,
    batch_size=128,
    validation_split=0.2
)
print("\n✅ Training resumed!")

RESUME INTERRUPTED TRAINING
📂 Loading checkpoint...
✅ Loaded!

🏋️ Resuming from epoch 3 to 6...
Epoch 4/6
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - accuracy: 0.8574 - loss: 0.3426 - val_accuracy: 0.8207 - val_loss: 0.3951
Epoch 5/6
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.8707 - loss: 0.3151 - val_accuracy: 0.8298 - val_loss: 0.3800
Epoch 6/6
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.8873 - loss: 0.2875 - val_accuracy: 0.8278 - val_loss: 0.3888

✅ Training resumed!


In [10]:
# =============================================================================
# CELL 10: EARLY STOPPING
# =============================================================================

print("="*60)
print("EARLY STOPPING")
print("="*60)

model_es = keras.Sequential([
    layers.Embedding(5000, 32, input_length=100),
    layers.LSTM(32, dropout=0.2, recurrent_dropout=0.2),
    layers.Dense(1, activation='sigmoid')
])
model_es.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=2,
    restore_best_weights=True,
    verbose=1
)

print("🏋️ Training with epochs=20 (early stopping will interrupt)...\n")
history_es = model_es.fit(
    x_train, y_train,
    epochs=20,
    batch_size=128,
    validation_split=0.2,
    callbacks=[early_stop]
)

actual_epochs = len(history_es.history['loss'])
print(f"\n✅ Stopped at epoch {actual_epochs} (saved {20 - actual_epochs} epochs!)")

EARLY STOPPING
🏋️ Training with epochs=20 (early stopping will interrupt)...

Epoch 1/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - accuracy: 0.6967 - loss: 0.5851 - val_accuracy: 0.8065 - val_loss: 0.4398
Epoch 2/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.8309 - loss: 0.3996 - val_accuracy: 0.8245 - val_loss: 0.3961
Epoch 3/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.8617 - loss: 0.3392 - val_accuracy: 0.8288 - val_loss: 0.3862
Epoch 4/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.8725 - loss: 0.3098 - val_accuracy: 0.8255 - val_loss: 0.3947
Epoch 5/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.8859 - loss: 0.2859 - val_accuracy: 0.8300 - val_loss: 0.3932
Epoch 5: early stopping
Restoring model weights from the end of the best epoch: 3.

✅ Stopped at epoch 5 (saved 15 epochs!)


In [11]:
# =============================================================================
# CELL 11: REDUCE LR ON PLATEAU
# =============================================================================

print("="*60)
print("REDUCE LEARNING RATE ON PLATEAU")
print("="*60)

model_lr = keras.Sequential([
    layers.Embedding(5000, 32, input_length=100),
    layers.LSTM(32, dropout=0.2, recurrent_dropout=0.2),
    layers.Dense(1, activation='sigmoid')
])
model_lr.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=1,
    min_lr=0.00001,
    verbose=1
)

print("🏋️ Training (watch 'reducing learning rate' messages)...\n")
model_lr.fit(
    x_train, y_train,
    epochs=6,
    batch_size=128,
    validation_split=0.2,
    callbacks=[reduce_lr]
)

REDUCE LEARNING RATE ON PLATEAU
🏋️ Training (watch 'reducing learning rate' messages)...

Epoch 1/6
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - accuracy: 0.6896 - loss: 0.6018 - val_accuracy: 0.8067 - val_loss: 0.4285 - learning_rate: 0.0010
Epoch 2/6
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.8240 - loss: 0.4118 - val_accuracy: 0.8257 - val_loss: 0.3931 - learning_rate: 0.0010
Epoch 3/6
124/125 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.8605 - loss: 0.3473
Epoch 3: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.8559 - loss: 0.3536 - val_accuracy: 0.8055 - val_loss: 0.4382 - learning_rate: 0.0010
Epoch 4/6
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.8724 - loss: 0.3160 - val_accuracy: 0.8372 - val_loss: 0.3848 - learning_rate: 5.0000e-04
Epoch 5/6
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.8855 - loss: 0.2909 - val_accuracy: 0.8432 - val_loss: 0.3809 - learning_rate: 5

In [12]:
# =============================================================================
# CELL 12: ALL CALLBACKS TOGETHER
# =============================================================================

print("="*60)
print("PRODUCTION SETUP: ALL CALLBACKS")
print("="*60)

model_prod = keras.Sequential([
    layers.Embedding(5000, 32, input_length=100),
    layers.LSTM(32, dropout=0.2, recurrent_dropout=0.2),
    layers.Dense(1, activation='sigmoid')
])
model_prod.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

callbacks_prod = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1),
    ModelCheckpoint('checkpoints/production_best.keras', monitor='val_accuracy', save_best_only=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=1, verbose=1)
]

print("🏋️ Training with all callbacks...\n")
model_prod.fit(
    x_train, y_train,
    epochs=15,
    batch_size=128,
    validation_split=0.2,
    callbacks=callbacks_prod
)

print("\n✅ Production training complete!")

PRODUCTION SETUP: ALL CALLBACKS
🏋️ Training with all callbacks...

Epoch 1/15
124/125 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.5738 - loss: 0.6652
Epoch 1: val_accuracy improved from None to 0.78725, saving model to checkpoints/production_best.keras

Epoch 1: finished saving model to checkpoints/production_best.keras
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - accuracy: 0.6639 - loss: 0.6000 - val_accuracy: 0.7872 - val_loss: 0.4486 - learning_rate: 0.0010
Epoch 2/15
123/125 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.8228 - loss: 0.4117
Epoch 2: val_accuracy improved from 0.78725 to 0.82650, saving model to checkpoints/production_best.keras

Epoch 2: finished saving model to checkpoints/production_best.keras
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.8271 - loss: 0.4019 - val_accuracy: 0.8265 - val_loss: 0.3898 - learning_rate: 0.0010
Epoch 3/15
124/125 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.8595 - loss: 0.3412
Epoch 3: val_accuracy improved from 0.826

In [13]:
# =============================================================================
# CELL 13: OVERFIT MODEL
# =============================================================================

print("="*60)
print("OVERFITTING DEMO: BIG MODEL, NO REGULARIZATION")
print("="*60)

model_overfit = keras.Sequential([
    layers.Embedding(5000, 128, input_length=100),
    layers.LSTM(128),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])
model_overfit.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Build model first to count parameters
model_overfit.build(input_shape=(None, 100))
print(f"📊 Parameters: {model_overfit.count_params():,}\n")

history_overfit = model_overfit.fit(
    x_train, y_train,
    epochs=5,
    batch_size=128,
    validation_split=0.2
)

train_acc = history_overfit.history['accuracy'][-1]
val_acc = history_overfit.history['val_accuracy'][-1]
gap = train_acc - val_acc

print(f"\n📊 Train: {train_acc:.1%} | Val: {val_acc:.1%} | Gap: {gap:.1%}")
if gap > 0.1:
    print("⚠️  Gap > 10% = OVERFITTING!")

OVERFITTING DEMO: BIG MODEL, NO REGULARIZATION
📊 Parameters: 779,905

Epoch 1/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 6s 47ms/step - accuracy: 0.7441 - loss: 0.5002 - val_accuracy: 0.8285 - val_loss: 0.3811
Epoch 2/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 6s 45ms/step - accuracy: 0.8714 - loss: 0.3067 - val_accuracy: 0.8455 - val_loss: 0.3712
Epoch 3/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 6s 45ms/step - accuracy: 0.9022 - loss: 0.2474 - val_accuracy: 0.8480 - val_loss: 0.3818
Epoch 4/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 5s 44ms/step - accuracy: 0.9259 - loss: 0.1907 - val_accuracy: 0.8447 - val_loss: 0.4487
Epoch 5/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 6s 44ms/step - accuracy: 0.9434 - loss: 0.1515 - val_accuracy: 0.8370 - val_loss: 0.4679

📊 Train: 94.3% | Val: 83.7% | Gap: 10.6%
⚠️  Gap > 10% = OVERFITTING!
